In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

from utils import get_request
from db_utils.base_utils import get_db_connection
from db_utils.the_odds_db_utils import *

c:\Users\bills\Documents\python\accurateshothelper\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2025-01-01'

In [3]:
fetch_and_store_nhl_games('2024-12-30', enable_logging=True)

2025-01-01 16:46:10,708 - root - INFO - Fetching and storing NHL events for date: 2024-12-30
2025-01-01 16:46:10,709 - root - INFO - Using historical API for date: 2024-12-30
2025-01-01 16:46:13,119 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 16:46:13,144 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 16:46:13,147 - root - INFO - Inserted/Updated 16 records into game_info table.
2025-01-01 16:46:13,147 - root - INFO - Completed fetching and storing events.


[{'id': '8295b240b0f6ff2fb05e2d538b8d6653',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'commence_time': '2024-12-31T00:00:00Z',
  'home_team': 'Florida Panthers',
  'away_team': 'New York Rangers'},
 {'id': 'd17c6ce4147ba6fa457097fb74596e72',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'commence_time': '2024-12-31T00:30:00Z',
  'home_team': 'Winnipeg Jets',
  'away_team': 'Nashville Predators'},
 {'id': 'd172475cd20920b7adfc712a44429759',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'commence_time': '2024-12-31T01:00:00Z',
  'home_team': 'Seattle Kraken',
  'away_team': 'Utah Hockey Club'},
 {'id': '83c00f54e27e7e92fcccd05c3c63b1ee',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'commence_time': '2024-12-31T17:30:00Z',
  'home_team': 'Washington Capitals',
  'away_team': 'Boston Bruins'},
 {'id': '4bc20ca48417b7fb9cfc968826411196',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'commence_time': '2024-12-31T18:00:00Z',
  '

In [4]:
get_nhl_events_from_db('2024-12-30', enable_logging=True)

2025-01-01 16:46:13,170 - root - INFO - Retrieving NHL events from DB for date: 2024-12-30
2025-01-01 16:46:13,171 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 16:46:13,307 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 16:46:13,309 - root - INFO - Retrieved 3 events from the database for date 2024-12-30.


[{'id': '8295b240b0f6ff2fb05e2d538b8d6653',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'home_team': 'Florida Panthers',
  'away_team': 'New York Rangers',
  'commence_time': datetime.datetime(2024, 12, 30, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'id': 'd17c6ce4147ba6fa457097fb74596e72',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'home_team': 'Winnipeg Jets',
  'away_team': 'Nashville Predators',
  'commence_time': datetime.datetime(2024, 12, 30, 18, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'id': 'd172475cd20920b7adfc712a44429759',
  'sport_key': 'icehockey_nhl',
  'sport_title': 'NHL',
  'home_team': 'Seattle Kraken',
  'away_team': 'Utah Hockey Club',
  'commence_time': datetime.datetime(2024, 12, 30, 19, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))}]

In [5]:
process_all_sog_markets('2024-12-21', enable_logging=True)

2025-01-01 16:46:14,935 - root - INFO - Processing all SOG markets for date: 2024-12-21
2025-01-01 16:46:14,935 - root - INFO - Retrieving NHL events from DB for date: 2024-12-21
2025-01-01 16:46:14,935 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 16:46:14,959 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 16:46:14,961 - root - INFO - Retrieved 11 events from the database for date 2024-12-21.
2025-01-01 16:46:14,962 - root - INFO - Processing game 5ab0c960460775673784e9b6b252099b scheduled for 2024-12-21T18:30:00Z
2025-01-01 16:46:14,963 - root - INFO - Processing SOG markets for game: 5ab0c960460775673784e9b6b252099b
2025-01-01 16:46:14,963 - root - INFO - Using historical odds for date: 2024-12-21T18:30:00Z
2025-01-01 16:46:20,918 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 16:46:

In [6]:
def run_process_all_sog_markets_for_month(year: int, month: int, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each day in the specified month.

    Args:
        year (int): The year of the month to process.
        month (int): The month to process (1-12).
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    import calendar
    from datetime import datetime

    # Get the number of days in the specified month
    num_days = calendar.monthrange(year, month)[1]

    for day in range(1, num_days + 1):
        # Format the date as 'YYYY-MM-DD'
        date_str = datetime(year, month, day).strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)

In [10]:
run_process_all_sog_markets_for_month(year= 2024, month= 11)

2025-01-01 22:08:14,231 - root - INFO - Processing all SOG markets for date: 2024-11-01
2025-01-01 22:08:14,232 - root - INFO - Retrieving NHL events from DB for date: 2024-11-01
2025-01-01 22:08:14,232 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:08:14,257 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:08:14,259 - root - INFO - Retrieved 0 events from the database for date 2024-11-01.
2025-01-01 22:08:14,260 - root - INFO - No games found for date 2024-11-01, attempting to fetch from API
2025-01-01 22:08:14,260 - root - INFO - Fetching and storing NHL events for date: 2024-11-01
2025-01-01 22:08:14,260 - root - INFO - Using historical API for date: 2024-11-01


Processing SOG markets for date: 2024-11-01


2025-01-01 22:08:14,972 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:08:14,997 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:08:14,999 - root - INFO - Inserted/Updated 16 records into game_info table.
2025-01-01 22:08:14,999 - root - INFO - Completed fetching and storing events.
2025-01-01 22:08:15,000 - root - INFO - Retrieving NHL events from DB for date: 2024-11-01
2025-01-01 22:08:15,000 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:08:15,026 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:08:15,028 - root - INFO - Retrieved 5 events from the database for date 2024-11-01.
2025-01-01 22:08:15,029 - root - INFO - Processing game d2e04b2771dc0196eaf4ca53300d4960 scheduled for 2024-11-01T23:08:00Z
2025-01-01 22:08:15,029 - root - INF

Processing SOG markets for date: 2024-11-02


2025-01-01 22:08:19,100 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:08:19,125 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:08:19,136 - root - INFO - Inserted/Updated 156 records for game 7e23d636d5555a0ff45a2a707eaeb3af
2025-01-01 22:08:19,137 - root - INFO - Completed processing SOG markets for game 7e23d636d5555a0ff45a2a707eaeb3af
2025-01-01 22:08:19,138 - root - INFO - Processing game 58f8025b4a1110c3e34efba838b3fad7 scheduled for 2024-11-02T17:00:00Z
2025-01-01 22:08:19,138 - root - INFO - Processing SOG markets for game: 58f8025b4a1110c3e34efba838b3fad7
2025-01-01 22:08:19,138 - root - INFO - Using historical odds for date: 2024-11-02T17:00:00Z
2025-01-01 22:08:19,760 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:08:19,783 - db_utils.base_utils - INFO - Successfully e

...

2025-01-01 22:10:46,697 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:46,722 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:10:46,725 - root - INFO - Inserted/Updated 14 records into game_info table.
2025-01-01 22:10:46,725 - root - INFO - Completed fetching and storing events.
2025-01-01 22:10:46,726 - root - INFO - Retrieving NHL events from DB for date: 2024-11-28
2025-01-01 22:10:46,726 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:46,859 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:10:46,861 - root - INFO - Retrieved 0 events from the database for date 2024-11-28.
2025-01-01 22:10:46,862 - root - WARNING - No games found for date 2024-11-28 even after fetching from API
2025-01-01 22:10:46,862 - root - INFO - Processing al

Processing SOG markets for date: 2024-11-29


2025-01-01 22:10:47,862 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:48,001 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:10:48,009 - root - INFO - Inserted/Updated 140 records for game 1403605c0466719bba35b56d45cc2c6d
2025-01-01 22:10:48,010 - root - INFO - Completed processing SOG markets for game 1403605c0466719bba35b56d45cc2c6d
2025-01-01 22:10:48,010 - root - INFO - Processing game 052b21a4a8933922bfa6b005df958687 scheduled for 2024-11-29T19:00:00Z
2025-01-01 22:10:48,010 - root - INFO - Processing SOG markets for game: 052b21a4a8933922bfa6b005df958687
2025-01-01 22:10:48,011 - root - INFO - Using historical odds for date: 2024-11-29T19:00:00Z
2025-01-01 22:10:48,549 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:48,574 - db_utils.base_utils - INFO - Successfully e

Processing SOG markets for date: 2024-11-30


2025-01-01 22:10:57,335 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:57,361 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:10:57,364 - root - INFO - Inserted/Updated 15 records into game_info table.
2025-01-01 22:10:57,364 - root - INFO - Completed fetching and storing events.
2025-01-01 22:10:57,365 - root - INFO - Retrieving NHL events from DB for date: 2024-11-30
2025-01-01 22:10:57,365 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:10:57,389 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:10:57,391 - root - INFO - Retrieved 10 events from the database for date 2024-11-30.
2025-01-01 22:10:57,392 - root - INFO - Processing game 4c105120f454d93b727285592f242857 scheduled for 2024-12-01T00:00:00Z
2025-01-01 22:10:57,392 - root - IN

In [7]:
def run_process_all_sog_markets_for_date_range(start_date: str, end_date: str, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each date within the specified date range.
    
    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    from datetime import datetime, timedelta

    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    if start_dt > end_dt:
        raise ValueError("start_date must be on or before end_date")

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)
        
        # Move to the next day
        current_dt += timedelta(days=1)

In [11]:
run_process_all_sog_markets_for_date_range(start_date='2024-10-04', end_date='2024-10-31')

2025-01-01 22:18:45,237 - root - INFO - Processing all SOG markets for date: 2024-10-04
2025-01-01 22:18:45,237 - root - INFO - Retrieving NHL events from DB for date: 2024-10-04
2025-01-01 22:18:45,238 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:18:45,268 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:18:45,272 - root - INFO - Retrieved 0 events from the database for date 2024-10-04.
2025-01-01 22:18:45,272 - root - INFO - No games found for date 2024-10-04, attempting to fetch from API
2025-01-01 22:18:45,273 - root - INFO - Fetching and storing NHL events for date: 2024-10-04
2025-01-01 22:18:45,273 - root - INFO - Using historical API for date: 2024-10-04


Processing SOG markets for date: 2024-10-04


2025-01-01 22:18:45,918 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:18:46,047 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:18:46,050 - root - INFO - Inserted/Updated 22 records into game_info table.
2025-01-01 22:18:46,051 - root - INFO - Completed fetching and storing events.
2025-01-01 22:18:46,051 - root - INFO - Retrieving NHL events from DB for date: 2024-10-04
2025-01-01 22:18:46,051 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:18:46,075 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:18:46,077 - root - INFO - Retrieved 0 events from the database for date 2024-10-04.
2025-01-01 22:18:46,078 - root - WARNING - No games found for date 2024-10-04 even after fetching from API
2025-01-01 22:18:46,079 - root - INFO - Processing al

Processing SOG markets for date: 2024-10-05


2025-01-01 22:18:46,750 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:18:46,774 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:18:46,784 - root - INFO - Inserted/Updated 156 records for game 8d7e2592786731cf1eddf700a77375b5
2025-01-01 22:18:46,784 - root - INFO - Completed processing SOG markets for game 8d7e2592786731cf1eddf700a77375b5
2025-01-01 22:18:46,785 - root - INFO - Completed processing all SOG markets for date 2024-10-05
2025-01-01 22:18:46,786 - root - INFO - Processing all SOG markets for date: 2024-10-06
2025-01-01 22:18:46,786 - root - INFO - Retrieving NHL events from DB for date: 2024-10-06
2025-01-01 22:18:46,786 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:18:46,809 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:18

...

2025-01-01 22:20:26,602 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:26,739 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:20:26,742 - root - INFO - Inserted/Updated 13 records into game_info table.
2025-01-01 22:20:26,742 - root - INFO - Completed fetching and storing events.
2025-01-01 22:20:26,743 - root - INFO - Retrieving NHL events from DB for date: 2024-10-29
2025-01-01 22:20:26,743 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:26,882 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:20:26,884 - root - INFO - Retrieved 7 events from the database for date 2024-10-29.
2025-01-01 22:20:26,885 - root - INFO - Processing game 3297a7cb75ea8a0cd27c68ce0a7ef3bd scheduled for 2024-10-29T23:08:00Z
2025-01-01 22:20:26,885 - root - INF

Processing SOG markets for date: 2024-10-30


2025-01-01 22:20:32,385 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:32,511 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:20:32,521 - root - INFO - Inserted/Updated 160 records for game cd732af52a4c55b1a1c53806fad68578
2025-01-01 22:20:32,521 - root - INFO - Completed processing SOG markets for game cd732af52a4c55b1a1c53806fad68578
2025-01-01 22:20:32,521 - root - INFO - Processing game 899d0dc22dbac52e33200a1323d9dc06 scheduled for 2024-10-30T23:30:00Z
2025-01-01 22:20:32,522 - root - INFO - Processing SOG markets for game: 899d0dc22dbac52e33200a1323d9dc06
2025-01-01 22:20:32,522 - root - INFO - Using historical odds for date: 2024-10-30T23:30:00Z
2025-01-01 22:20:33,161 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:33,185 - db_utils.base_utils - INFO - Successfully e

Processing SOG markets for date: 2024-10-31


2025-01-01 22:20:36,282 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:36,417 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:20:36,419 - root - INFO - Inserted/Updated 13 records into game_info table.
2025-01-01 22:20:36,420 - root - INFO - Completed fetching and storing events.
2025-01-01 22:20:36,420 - root - INFO - Retrieving NHL events from DB for date: 2024-10-31
2025-01-01 22:20:36,420 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=the_odds, user=postgres
2025-01-01 22:20:36,451 - db_utils.base_utils - INFO - Successfully established database connection
2025-01-01 22:20:36,454 - root - INFO - Retrieved 7 events from the database for date 2024-10-31.
2025-01-01 22:20:36,455 - root - INFO - Processing game fe044d4b2228d62f28f0a0be82684e3b scheduled for 2024-10-31T23:08:00Z
2025-01-01 22:20:36,456 - root - INF